# Model comparison
This notebook evaluates a model according to an agreed upon metric in order to compare results with the GP models.

In [1]:
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt

In [45]:
"""
Some loss functions
"""

def mae(preds, labels):
    mae = np.sum(np.absolute(preds - labels))
    print(mae)
    mae /= len(preds)
    print(mae)
    return mae

def basic_mae(preds, labels):
    acc = 0 
    n = len(preds)
    for i in range(n):
        e = abs(preds[i] - labels[i])
        acc += e
 
    acc /= n
    return [acc]

def mape(preds, labels):
    acc = 0
    n = len(preds)
    for i in range(n):
        acc+=abs((labels[i] - preds[i])/labels[i])
    
    res = acc/n
    return [res]

Pick the name for your model

In [3]:
model_name = 'ANN_M2'

In [4]:
data = pds.read_pickle(model_name + '.pkl')
data.head()

,seg,journey,speed,pred,label
0,1,4,0.0,122.613525,187.0
1,1,4,0.0,122.613525,186.0
2,1,4,0.0,122.613525,185.0
3,1,4,0.0,122.613525,184.0
4,1,4,0.0,122.613525,183.0


In [61]:
jour_frames = []

for ij, jour in data.groupby('journey'):
    seg_frames = []
    
    for iss, seg in jour.groupby('seg'):
        tot = len(seg) - 1
        errs = {}
        i_20 = np.int_(np.round(tot * 0.2))
        i_40 = np.int_(np.round(tot * 0.4))
        i_60 = np.int_(np.round(tot * 0.6))
        i_80 = np.int_(np.round(tot * 0.8))
        
        p_20 = seg.iloc[0:i_20]
        p_40 = seg.iloc[0:i_40]
        p_60 = seg.iloc[0:i_60]
        p_80 = seg.iloc[0:i_80]
        errs['journey'] = [ij]
        errs['segment'] = [iss]
        
        errs['e_20'] = basic_mae(p_20.pred.values, p_20.label.values)
        errs['ep_20'] = mape(p_20.pred.values, p_20.label.values)
        
        errs['e_40'] = basic_mae(p_40.pred.values, p_40.label.values)
        errs['ep_40'] = mape(p_40.pred.values, p_40.label.values)
        
        errs['e_60'] = basic_mae(p_60.pred.values, p_60.label.values)
        errs['ep_60'] = mape(p_60.pred.values, p_60.label.values)
        
        errs['e_80'] = basic_mae(p_80.pred.values, p_80.label.values)
        errs['ep_80'] = mape(p_80.pred.values, p_80.label.values)
        
        seg_frames.append(pds.DataFrame(errs))
    
    jour_frames.append(pds.concat(seg_frames))
        

In [62]:
ers = pds.concat(jour_frames).reset_index(drop=True)

In [63]:
ers.head(30)

,journey,segment,e_20,ep_20,e_40,ep_40,e_60,ep_60,e_80,ep_80
0,4,1,46.386475,0.271559,28.749880,0.176652,28.704834,0.224546,26.326244,0.241322
1,4,2,4.534348,0.092543,6.702279,0.165648,5.617040,0.153775,4.935362,0.157308
2,4,3,1.812978,0.061058,1.234046,0.043771,0.849781,0.031235,0.759788,0.034650
3,4,4,1.503548,0.071490,1.396813,0.073784,1.172122,0.070200,1.024835,0.070091
4,4,5,2.192088,0.048640,1.420669,0.033795,1.287798,0.036574,1.794043,0.079333
5,4,6,3.256615,0.060586,5.342701,0.113491,3.956244,0.087093,3.979876,0.127077
6,4,7,3.142668,0.037479,6.599669,0.093750,4.957781,0.074257,4.234251,0.075736
7,4,8,1.946607,0.038720,4.507762,0.113219,3.798199,0.103303,3.189853,0.098462
8,4,9,4.621133,0.034429,4.434001,0.038271,3.801066,0.036595,3.714606,0.048014
9,4,10,15.657351,0.254135,16.741435,0.305501,12.111152,0.226508,9.376708,0.186099


In [64]:
ers.mean(axis=0)

journey    1769.097561
segment       6.000000
e_20         12.667052
ep_20         0.139794
e_40         10.755603
ep_40         0.145446
e_60          9.621309
ep_60         0.149449
e_80          8.654606
ep_80         0.161229
dtype: float64

In [65]:
ers[ers.segment != 1].mean(axis=0)

journey    1769.097561
segment       6.500000
e_20          8.780195
ep_20         0.125466
e_40          8.225852
ep_40         0.138359
e_60          7.199716
ep_60         0.138985
e_80          6.217390
ep_80         0.146317
dtype: float64